In [10]:
import numpy as np
import xtrack as xt
import xobjects as xo
import xpart as xp
import numpy as np
import matplotlib.pyplot as plt

In [11]:
#lattice parameters for ELENA at 100 keV
# from https://acc-models.web.cern.ch/acc-models/elena/scenarios/highenergy/highenergy.tfs
qx = 2.36168984503
qy = 1.38992572490
circumference = 30.40531277976 #m

# relativistic factors
gamma_rel = 1.0001066 # at 100 keV

# optics at e-cooler (approximate), in m
beta_x = 1.7
beta_y = 2.7
D_x = 1

# electron cooler parameters
current = 0.001 # A current
length = 1 # m cooler length
radius_e_beam = 25*1e-3 #m radius of the electron beam
temp_perp = 100e-3 # <E> [eV] = kb*T
temp_long =  1e-3 # <E> [eV]
magnetic_field = 0.010 # 100 Gauss in ELENA
# idea is to study magnetic field imperfections
magnetic_field_ratio_list = [0,5e-4,1e-3,5e-3] #Iterate over different values of the magnetic field quality to see effect on cooling performance.
#magnetic_field_ratio is the ratio of transverse componenet of magnetic field and the longitudinal component. In the ideal case, the ratio is 0.

# some initial beam parameters
emittance = 5e-6
dp_p = 2e-3 

# simulation parameters: simulate 10 s of cooling, and take data once every 10 ms
max_time_s = 10
int_time_s = 0.01

qs=0.007718714437902285
bets0=-469.32883416451523

arc_matching = xt.LineSegmentMap(
        qx=qx, qy=qy,
        dqx=0, dqy=0,
        length=circumference,
        betx=beta_x,
        bety=beta_y,
        dx=0,
        qs=qs,
        bets=bets0)

line_matching=xt.Line([arc_matching])
line_matching.build_tracker()

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [12]:
# some constants, and simple computations
clight = 299792458.0
mass0 = 938.27208816*1e6 #ev/c^2

beta_rel = np.sqrt(gamma_rel**2 - 1)/gamma_rel
p0c = mass0*beta_rel*gamma_rel #eV/c
T_per_turn = circumference/(clight*beta_rel)

# compute length of simulation, as well as sample interval, in turns
num_turns = int(max_time_s/T_per_turn)
save_interval = int(int_time_s/T_per_turn)

# # compute initial beam parameters
# x_init = np.sqrt(beta_x*emittance)
# y_init = np.sqrt(beta_y*emittance)


In [13]:
from tqdm import tqdm

n_part = 10000

data=np.load('H_bumps/bumps.npz')

n_steps=data['n_steps']
num_samples=data['num_samples']
delay_list=data['delay_list']
repeated_delay=data['repeated_delay']
bumps_list=data['bumps_list']


min_nonzero_value = np.min(repeated_delay[repeated_delay != 0])
simulation_time=(np.max(repeated_delay)-min_nonzero_value)/1000

# bump_list=np.linspace(-40e-3,40e-3,50)

final_emittance_x=[]
final_emittance_x_normalised=[]
sigma_x_list=[]

final_emittance_y=[]
final_emittance_y_normalised=[]
sigma_y_list=[]

# simulation parameters: simulate 10 s of cooling, and take data once every 10 ms
max_time_s = simulation_time
int_time_s = 0.01

# compute length of simulation, as well as sample interval, in turns
num_turns = int(max_time_s/T_per_turn)
save_interval = int(int_time_s/T_per_turn)


#plot some overall values

arc = xt.LineSegmentMap(
                qx=qx, qy=qx,
                dqx=0, dqy=0,
                length=circumference,
                betx=beta_x,
                bety=beta_y,
                dx=D_x)


for magnetic_ratio in tqdm(magnetic_field_ratio_list):
        # Define the whole machine
        
        electron_cooler = xt.ElectronCooler(
                length=length,
                radius_e_beam=radius_e_beam,
                current=current,
                temp_perp=temp_perp,
                temp_long=temp_long,
                magnetic_field=magnetic_field, 
                magnetic_field_ratio=magnetic_ratio,
                space_charge=1)

        # create a monitor object, to reduce holded data
        monitor = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=1,
                                n_repetitions=int(num_turns/save_interval),
                                repetition_period=save_interval,
                                num_particles=n_part)

        line = xt.Line(
                elements=[monitor, electron_cooler, arc])
        line.particle_ref = xp.Particles(mass0=mass0, q0=1, p0c=p0c)
        context = xo.ContextCpu(omp_num_threads=6)
        line.build_tracker(_context=context)

        # create desired beam
        bunch_intensity = None
        beta_gamma = line.particle_ref._beta0*line.particle_ref._gamma0
        gemitt_x = 10e-6
        gemitt_y = 5e-6
        nemitt_x = gemitt_x*beta_gamma
        nemitt_y = gemitt_y*beta_gamma
        sigma_dp = 1e-3
        # particles = xp.generate_matched_gaussian_bunch(
        #         num_particles=n_part, total_intensity_particles=bunch_intensity,
        #         nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_dp=sigma_dp,
        #         coasting=True,
        #         line=line)

        particles = xp.generate_matched_gaussian_bunch(
                num_particles=n_part,
                # total_intensity_particles=bunch_intensity,
                nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_z = 0.3284, # in m,
                particle_ref=line.particle_ref ,
                line=line_matching,        
        )

        sigma_x = np.sqrt(beta_x*emittance)
        sigma_px = np.sqrt(emittance*1/beta_x)
        sigma_y = np.sqrt(beta_y*emittance)
        sigma_py = np.sqrt(emittance*1/beta_y)
        sigma_p = 5e-3

        delta = np.random.uniform(-sigma_dp,sigma_dp,n_part)
        # x = np.random.normal(loc=0.0, scale=sigma_x, size=n_part) + D_x * delta
        # px = np.random.normal(loc=0.0, scale=sigma_px, size=n_part)
        # y = np.random.normal(loc=0.0, scale=sigma_y, size=n_part)
        # py = np.random.normal(loc=0.0, scale=sigma_py, size=n_part)

        # particles = xp.Particles(
        # mass0=mass0,
        # p0c=p0c,
        # q0=1,
        # x=x,
        # px=px,
        # y=y,
        # py=py,
        # delta=delta,
        # zeta=0
        # )

        particles.zeta = np.random.uniform(-circumference/2, circumference/2, n_part)

        # just track all particles, and keep turn-by-turn data (memory expensive!)
        line.track(particles, num_turns=num_turns,
                turn_by_turn_monitor=False)

        # extract relevant values
        x = monitor.x[:,:,0]
        px = monitor.px[:,:,0]
        y = monitor.y[:,:,0]
        py = monitor.py[:,:,0]
        delta = monitor.delta[:,:,0]
        zeta = monitor.zeta[:,:,0]
        time = monitor.at_turn[:, 0, 0] * T_per_turn

        # compute actions. for x, remove the dp/p contribution:
        action_x = ((x-D_x*delta)**2/beta_x + beta_x*px**2)
        # for y, simple compute:
        action_y = (y**2/beta_y + beta_y*py**2)

        norm_emittance_x=np.mean(action_x,axis=1)/2*gamma_rel*beta_rel
        norm_emittance_y=np.mean(action_y,axis=1)/2*gamma_rel*beta_rel

        geo_emittance_x=np.mean(action_x,axis=1)/2
        geo_emittance_y=np.mean(action_y,axis=1)/2


        np.savez(f'results/magnetic_field_imperfection/magnetic_ratio_{magnetic_ratio}.npz',
                
                # emittance_x_twiss=emittance_x_twiss,
                # emittance_y_twiss=emittance_y_twiss,
                geo_emittance_x=geo_emittance_x,
                geo_emittance_y=geo_emittance_y,
                norm_emittance_x=norm_emittance_x,
                norm_emittance_y=norm_emittance_y,
                x=x,  
                y=y,  
                px=px, 
                py=py,
                delta=delta,
                zeta=zeta,
                time=time)  

  0%|          | 0/4 [00:00<?, ?it/s]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


 25%|██▌       | 1/4 [01:39<04:57, 99.24s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


 50%|█████     | 2/4 [03:13<03:13, 96.55s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


 75%|███████▌  | 3/4 [04:51<01:36, 96.96s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


100%|██████████| 4/4 [06:28<00:00, 97.15s/it]


In [14]:
beta_gamma

array([0.01460176])